# Potencia Aparente Posiciones

 - **Desarrollador:** Víctor Echeverría
 - **Fecha:** 2020-09-12
 - **Descripción:** Se realiza la carga de los datos para realizar las ejecuciones calendarizadas de los ETLs de Potencia Aparente de Posiciones y carga inicial de datos.

In [1]:
from ExceptionManager import ExceptionManager
from HDFSContext import HDFSContext
from GenericDataFrame import GenericDataFrame
from DBContextDw import DBContextDw
from EtlAL import EtlAL
from LimpiarPosiciones import LimpiarPosiciones

from pyspark.sql import functions as Funct
from pyspark.sql.functions import regexp_replace,row_number, col,upper,udf
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import timedelta
import datetime
import time
from dateutil.relativedelta import relativedelta

In [2]:
dbContext = DBContextDw(Database='dwh_sirio',urlDriver='/home/jovyan/work/postgresql-42.2.12.jar')
accesoDatos = EtlAL(dbContext)
spark = dbContext.spark

def EstablecerPeriodosEjecucion(modulo): 
    """Método que inserta datos de los periodos a ser procesados"""
    try: 

        maxPk = accesoDatos.GetMaxPkTable("select max(tmp_etl_id_pk) pk from cen_cnt.tmp_etl_fecha_carga")
        periodos = []

        tmp_etl_fecha_carga = accesoDatos.GetAllData("cen_cnt.tmp_etl_fecha_carga")\
        .where(col('tmp_etl_modulo') == modulo)

        etl_fecha_carga = accesoDatos.GetAllData("cen_cnt.etl_fecha_carga")\
        .where((col('etl_modulo') == modulo) &\
               (col('etl_forzar') == False)).first() 

        if etl_fecha_carga is not None:
            modulo = etl_fecha_carga.etl_modulo
            fecha_inicio = etl_fecha_carga.etl_fecha_inicio
            fecha_fin_tot = etl_fecha_carga.etl_fecha_fin
            while (fecha_inicio < fecha_fin_tot):
                fecha_fin = fecha_inicio + timedelta(minutes=1439)
                periodos.append([modulo,fecha_inicio, fecha_fin,int(fecha_inicio.strftime('%Y%m%d')),
                                 int(fecha_fin.strftime('%Y%m%d'))])
                fecha_inicio = fecha_inicio + timedelta(days=1)

        schema = StructType([
            StructField("tmp_etl_modulo", StringType(),False),
            StructField("tmp_etl_fecha_inicio", TimestampType(),False),
            StructField("tmp_etl_fecha_fin", TimestampType(),False),
            StructField("tmp_etl_id_fecha_inicio", LongType(),False),
            StructField("tmp_etl_id_fecha_fin", LongType(),False)])
        df_periodos =  spark.createDataFrame(periodos,schema=schema)
        df_periodos = df_periodos.select((maxPk+row_number()\
                                          .over(Window.partitionBy().orderBy('tmp_etl_fecha_inicio')))\
                                         .alias("tmp_etl_id_pk"),'*')

        salida = accesoDatos.Insert(df_periodos, "cen_cnt.tmp_etl_fecha_carga")

        return salida

    except Exception as error:
        ExceptionManager.Treatment(error)
    
def Procesar(fecha_inicio,fecha_fin,id_fecha_inicio,id_fecha_fin,modulo,deteleTmpQuery):
    salida = LimpiarPosiciones.Procesamiento(fecha_inicio,fecha_fin,modulo,deteleTmpQuery)
    return (id_fecha_inicio,id_fecha_fin,salida)
            
def EjecutarETL(modulo):
    """Método que realiza la carga incremental"""
    try: 
        print("============================= CARGA INCREMENTAL DE DATOS ===================================")
        
        tmp_etl_fecha_carga = accesoDatos.GetAllData('cen_cnt.tmp_etl_fecha_carga')\
        .where(col('tmp_etl_modulo') == modulo)


        if(tmp_etl_fecha_carga is None):
            print ("\n **** No existen periodos a ejecutar.")
            return None

        fecha_inicio = tmp_etl_fecha_carga.groupby().agg(Funct.min('tmp_etl_fecha_inicio'),
                                                         Funct.min('tmp_etl_id_fecha_inicio'))
        fecha_fin = tmp_etl_fecha_carga.groupby().agg(Funct.max('tmp_etl_fecha_fin'),
                                                      Funct.max('tmp_etl_id_fecha_fin'))

        periodo = "    ### Periodo: Desde => {0}   Hasta => {1}".format(fecha_inicio.first()[0], fecha_fin.first()[0])
        print(str(datetime.datetime.now()))
        print(periodo)
        deteleTmpQuery = 'DELETE FROM cen_cnt.tmp_etl_fecha_carga WHERE tmp_etl_fecha_inicio = \'{0}\' '\
                       'and tmp_etl_fecha_fin = \'{1}\' and tmp_etl_modulo = \'{2}\' '

        deteleFactQuery = 'DELETE FROM cen_dws.fact_posicion WHERE tmpo_id_fk between {0} and {1}'\
        .format(fecha_inicio.first()[1],fecha_fin.first()[1])


        transactDeleteFact = accesoDatos.Delete(deteleFactQuery)
        
        if transactDeleteFact == True: 
            procesamiento = tmp_etl_fecha_carga.rdd.map(lambda x: Procesar(x.tmp_etl_fecha_inicio,
                                                                           x.tmp_etl_fecha_fin,
                                                                           x.tmp_etl_id_fecha_inicio,
                                                                           x.tmp_etl_id_fecha_fin,
                                                                           modulo,
                                                                           deteleTmpQuery))
            for d in procesamiento.collect():
                print(d)
        print ("\n **** EXITOSO: Se ha procesado todos los periodos del módulo {0}".format(modulo))
        print(str(datetime.datetime.now()))
    except Exception as error:
        ExceptionManager.Treatment(error)

In [3]:
EstablecerPeriodosEjecucion('POTENCIA APARENTE POSICIONES')
EjecutarETL('POTENCIA APARENTE POSICIONES')

============================= CARGA INCREMENTAL DE DATOS ===================================
2020-09-12 08:01:27.043860
    ### Periodo: Desde => 2019-10-10 00:00:00   Hasta => 2019-10-31 23:59:00
(20191010, 20191010, True)
(20191011, 20191011, True)
(20191012, 20191012, True)
(20191013, 20191013, True)
(20191014, 20191014, True)
(20191015, 20191015, True)
(20191016, 20191016, True)
(20191017, 20191017, True)
(20191018, 20191018, True)
(20191019, 20191019, True)
(20191020, 20191020, True)
(20191021, 20191021, True)
(20191022, 20191022, True)
(20191023, 20191023, True)
(20191024, 20191024, True)
(20191025, 20191025, True)
(20191026, 20191026, True)
(20191027, 20191027, True)
(20191028, 20191028, True)
(20191029, 20191029, True)
(20191030, 20191030, True)
(20191031, 20191031, True)

 **** EXITOSO: Se ha procesado todos los periodos del módulo POTENCIA APARENTE POSICIONES
2020-09-12 08:07:03.869744
